In [12]:
import glob
import skimage.io as io
import numpy as np
#import commonfunctions as cf
import sklearn
from skimage.feature import hog
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import pickle
import cv2
import random
import os
from PIL import Image, ImageEnhance
from PIL import Image, ImageEnhance
import cv2
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict, RandomizedSearchCV
from sklearn.metrics import *
from sklearn.decomposition import PCA
import pickle
import pandas as pd



In [5]:
male_data=np.load('male_data_small.npy',allow_pickle=True)
female_data = np.load('female_data_small.npy',allow_pickle=True)

male_data_icdar=np.load('male_data_icdar.npy',allow_pickle=True)
female_data_icdar=np.load('female_data_icdar.npy',allow_pickle=True)


In [22]:

male_data_combined=np.concatenate((male_data,male_data_icdar[:186]))
female_data_combined=np.concatenate((female_data,female_data_icdar))
print(male_data_combined.shape)
print(female_data_combined.shape)


(418, 1420, 1000)
(417, 1420, 1000)


In [7]:
from skimage import feature, exposure

def HistogramofOrientedGradients(image):
    resized_img = resize(image, (64, 128))
    fd, hog_image = feature.hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True)
    
    return (np.array(fd)).flatten()



In [20]:


N_ANGLE_BINS = 40
BIN_SIZE = 360 // N_ANGLE_BINS
LEG_LENGTH = 25

def get_contour_pixels(bw_image):
    contours, _= cv2.findContours(
        bw_image, cv2.RETR_TREE, 
        cv2.CHAIN_APPROX_NONE
        ) 
    # contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[1:]
    
    img2 = bw_image.copy()[:,:,np.newaxis]
    img2 = np.concatenate([img2, img2, img2], axis = 2)
    
    # if self.show_images:
    #     for cnt in contours : 
    #         cv2.drawContours(img2, [cnt], 0, (255, 0, 0), 1)  
            
    #     plt.imshow(img2, cmap='gray')
    return contours

def get_hinge_features(bw_image):
    # if self.is_binary:
    #     bw_image, _ = self.preprocess_binary_image(img_file, self.sharpness_factor, self.bordersize)
    # else:
    #     bw_image, _ = self.preprocess_image(img_file, self.sharpness_factor, self.bordersize)
    
    contours = get_contour_pixels(bw_image)
    
    hist = np.zeros((N_ANGLE_BINS, N_ANGLE_BINS))
        
    # print([len(cnt) for cnt in contours])
    for cnt in contours:
        n_pixels = len(cnt)
        if n_pixels <= LEG_LENGTH:
            continue
        
        points = np.array([point[0] for point in cnt])
        xs, ys = points[:, 0], points[:, 1]
        point_1s = np.array([cnt[(i + LEG_LENGTH) % n_pixels][0] for i in range(n_pixels)])
        point_2s = np.array([cnt[(i - LEG_LENGTH) % n_pixels][0] for i in range(n_pixels)])
        x1s, y1s = point_1s[:, 0], point_1s[:, 1]
        x2s, y2s = point_2s[:, 0], point_2s[:, 1]
        
        phi_1s = np.degrees(np.arctan2(y1s - ys, x1s - xs) + np.pi)
        phi_2s = np.degrees(np.arctan2(y2s - ys, x2s - xs) + np.pi)
        
        indices = np.where(phi_2s > phi_1s)[0]
        
        for i in indices:
            phi1 = int(phi_1s[i] // BIN_SIZE) % N_ANGLE_BINS
            phi2 = int(phi_2s[i] // BIN_SIZE) % N_ANGLE_BINS
            hist[phi1, phi2] += 1
            
    normalised_hist = hist / np.sum(hist)
    feature_vector = normalised_hist[np.triu_indices_from(normalised_hist, k = 1)]
    
    return feature_vector


#Initialization:
N_RHO_BINS = 7
N_ANGLE_BINS = 12
N_BINS = N_RHO_BINS * N_ANGLE_BINS
BIN_SIZE = 360 // N_ANGLE_BINS
R_INNER = 5.0
R_OUTER = 35.0
K_S = np.arange(3, 8)
def get_cold_features(bw_image, approx_poly_factor = 0.01 , sharpness_factor = 10 , bordersize = 3):

    #bw_image, _ = preprocess_image(img_file, sharpness_factor, bordersize)
    contours = get_contour_pixels(bw_image)
    
    rho_bins_edges = np.log10(np.linspace(R_INNER, R_OUTER, N_RHO_BINS))
    feature_vectors = np.zeros((len(K_S), N_BINS))
    
    for j, k in enumerate(K_S):
        hist = np.zeros((N_RHO_BINS, N_ANGLE_BINS))
        for cnt in contours:
            epsilon = approx_poly_factor * cv2.arcLength(cnt,True)
            cnt = cv2.approxPolyDP(cnt,epsilon,True)
            n_pixels = len(cnt)
            
            point_1s = np.array([point[0] for point in cnt])
            x1s, y1s = point_1s[:, 0], point_1s[:, 1]
            point_2s = np.array([cnt[(i + k) % n_pixels][0] for i in range(n_pixels)])
            x2s, y2s = point_2s[:, 0], point_2s[:, 1]
            
            thetas = np.degrees(np.arctan2(y2s - y1s, x2s - x1s) + np.pi)
            rhos = np.sqrt((y2s - y1s) ** 2 + (x2s - x1s) ** 2)
            if(rhos.all()):
                rhos_log_space = np.log10(rhos)
            else:
                rhos_log_space = rhos
            
            quantized_rhos = np.zeros(rhos.shape, dtype=int)
            for i in range(N_RHO_BINS):
                quantized_rhos += (rhos_log_space < rho_bins_edges[i])
                
            for i, r_bin in enumerate(quantized_rhos):
                theta_bin = int(thetas[i] // BIN_SIZE) % N_ANGLE_BINS
                hist[r_bin - 1, theta_bin] += 1
            
        normalised_hist = hist / hist.sum()
        feature_vectors[j] = normalised_hist.flatten()
        
    return feature_vectors.flatten()

def get_hinge_with_cold(bw_image):
    hinge=get_hinge_features(bw_image)
    cold=get_cold_features(bw_image)
    return np.concatenate([hinge, cold])
    

In [15]:

def make_model(male_data,female_data,get_feature,model_name,pca_flag=False,pca_number=50):
    male_feature=[]
    female_feature=[]
    for i,img in enumerate(male_data):
        male_feature.append(get_feature(img))

    for i,img in enumerate(female_data):        
        female_feature.append(get_feature(img))

    
    male_feature=np.array(male_feature)
    female_feature=np.array(female_feature)

    X=np.concatenate((male_feature,female_feature),axis=0)
    Y=np.concatenate((np.ones(male_feature.shape[0]),
                        np.zeros(female_feature.shape[0])),
                        axis=0)
    if pca_flag:
        pca=PCA(n_components=pca_number)
        X=pca.fit_transform(X)

    
    X,Y=sklearn.utils.shuffle(X,Y)
    #X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.01,random_state=42) # 70% training and 30% test

    model = svm.SVC(kernel='rbf', verbose=True, C=10,probability=True)

    model.fit(X, Y)
    
    #y_pred_cold=model.predict(X_test)
    #y_pred_prob_cold=model.predict_proba(X_test)
    #print(y_pred_prob_cold)
    #print("Accuracy:",metrics.accuracy_score(y_test, y_pred_cold))
    pickle.dump(model, open(model_name, 'wb'))

    
        

In [14]:
make_model(male_data,female_data,get_cold_features,"cold",True,50)

[LibSVM]..*.*
optimization finished, #iter = 978
obj = -359.830312, rho = -0.937492
nSV = 180, nBSV = 8
Total nSV = 180
..*.*
optimization finished, #iter = 974
obj = -368.038414, rho = -0.512505
nSV = 190, nBSV = 12
Total nSV = 190
..*.*
optimization finished, #iter = 963
obj = -374.091734, rho = -0.590867
nSV = 186, nBSV = 12
Total nSV = 186
..*.*
optimization finished, #iter = 912
obj = -353.463477, rho = -0.601134
nSV = 177, nBSV = 11
Total nSV = 177
..*.*
optimization finished, #iter = 903
obj = -334.735343, rho = -0.684755
nSV = 179, nBSV = 9
Total nSV = 179
..*.*
optimization finished, #iter = 1096
obj = -438.583198, rho = 0.728311
nSV = 210, nBSV = 14
Total nSV = 210


In [16]:
make_model(male_data,female_data,get_hinge_features,"hinge",False)

[LibSVM].*.*
optimization finished, #iter = 717
obj = -1216.622500, rho = -2.620886
nSV = 195, nBSV = 123
Total nSV = 195
.*
optimization finished, #iter = 568
obj = -1184.358176, rho = -2.008217
nSV = 187, nBSV = 125
Total nSV = 187
.*.*
optimization finished, #iter = 846
obj = -1107.545181, rho = -2.038007
nSV = 186, nBSV = 106
Total nSV = 186
.*.*
optimization finished, #iter = 821
obj = -1214.044620, rho = -1.854121
nSV = 198, nBSV = 118
Total nSV = 198
.*.*
optimization finished, #iter = 711
obj = -1181.604039, rho = -2.041304
nSV = 189, nBSV = 119
Total nSV = 189
.*.*
optimization finished, #iter = 893
obj = -1478.424261, rho = 1.987140
nSV = 232, nBSV = 153
Total nSV = 232


In [17]:
make_model(male_data,female_data, HistogramofOrientedGradients,"hog",False)

[LibSVM].*.*
optimization finished, #iter = 665
obj = -249.693048, rho = -0.515540
nSV = 276, nBSV = 0
Total nSV = 276
.*.*
optimization finished, #iter = 636
obj = -246.952474, rho = -0.621603
nSV = 278, nBSV = 0
Total nSV = 278
.*.*
optimization finished, #iter = 658
obj = -257.898202, rho = -0.859037
nSV = 277, nBSV = 0
Total nSV = 277
.*.*
optimization finished, #iter = 663
obj = -250.110807, rho = -0.656442
nSV = 278, nBSV = 0
Total nSV = 278
.*.*
optimization finished, #iter = 626
obj = -244.142445, rho = -0.864131
nSV = 270, nBSV = 0
Total nSV = 270
.*.*
optimization finished, #iter = 810
obj = -316.868343, rho = 0.767954
nSV = 341, nBSV = 0
Total nSV = 341


In [21]:
make_model(male_data,female_data,get_hinge_with_cold,"hinge_with_cold",True,70)

[LibSVM]..*.*
optimization finished, #iter = 928
obj = -307.026619, rho = -0.815315
nSV = 189, nBSV = 4
Total nSV = 189
..*
optimization finished, #iter = 853
obj = -275.434910, rho = -0.555302
nSV = 181, nBSV = 7
Total nSV = 181
..*.*
optimization finished, #iter = 933
obj = -313.568779, rho = -0.492849
nSV = 196, nBSV = 6
Total nSV = 196
..*.*
optimization finished, #iter = 875
obj = -307.243546, rho = -0.625049
nSV = 188, nBSV = 7
Total nSV = 188
..*.*
optimization finished, #iter = 942
obj = -300.281169, rho = -0.892592
nSV = 195, nBSV = 4
Total nSV = 195
..*.*
optimization finished, #iter = 1133
obj = -363.698857, rho = 0.742927
nSV = 229, nBSV = 9
Total nSV = 229


In [24]:
make_model(male_data_combined,female_data_combined,get_cold_features,"cold_combined",True,50)

[LibSVM]..*..*
optimization finished, #iter = 2895
obj = -1686.652563, rho = -0.698731
nSV = 451, nBSV = 125
Total nSV = 451
..*..*
optimization finished, #iter = 2715
obj = -1703.334844, rho = -0.545508
nSV = 451, nBSV = 127
Total nSV = 451
..*.*
optimization finished, #iter = 2549
obj = -1572.820572, rho = -0.617619
nSV = 427, nBSV = 111
Total nSV = 427
..*.*
optimization finished, #iter = 2490
obj = -1559.627072, rho = -0.572408
nSV = 431, nBSV = 112
Total nSV = 431
..*.*
optimization finished, #iter = 2605
obj = -1721.877669, rho = -0.614277
nSV = 441, nBSV = 128
Total nSV = 441
..*..*
optimization finished, #iter = 3650
obj = -2110.876181, rho = 0.607441
nSV = 547, nBSV = 156
Total nSV = 547


In [25]:
make_model(male_data_combined,female_data_combined,get_hinge_features,"hinge_combined",False)

[LibSVM].*.*
optimization finished, #iter = 1787
obj = -3400.675937, rho = -2.738917
nSV = 470, nBSV = 344
Total nSV = 470
.*.*
optimization finished, #iter = 1689
obj = -3368.135728, rho = -2.326981
nSV = 473, nBSV = 351
Total nSV = 473
.*.*
optimization finished, #iter = 1521
obj = -3380.725844, rho = -1.978955
nSV = 465, nBSV = 346
Total nSV = 465
.*.*
optimization finished, #iter = 1586
obj = -3355.656825, rho = -2.074608
nSV = 461, nBSV = 350
Total nSV = 461
.*.*
optimization finished, #iter = 1635
obj = -3274.664076, rho = -1.849740
nSV = 453, nBSV = 331
Total nSV = 453
.*.*
optimization finished, #iter = 2054
obj = -4220.140859, rho = 2.242158
nSV = 577, nBSV = 439
Total nSV = 577


In [26]:
make_model(male_data_combined,female_data_combined, HistogramofOrientedGradients,"hog_combined",False)

[LibSVM].*.*
optimization finished, #iter = 1698
obj = -712.614730, rho = -0.756568
nSV = 635, nBSV = 0
Total nSV = 635
.*.*
optimization finished, #iter = 1752
obj = -750.392186, rho = -0.878135
nSV = 641, nBSV = 0
Total nSV = 641
.*.*
optimization finished, #iter = 1733
obj = -746.051461, rho = -1.117814
nSV = 636, nBSV = 1
Total nSV = 636
.*.*
optimization finished, #iter = 1798
obj = -766.260864, rho = -0.718146
nSV = 647, nBSV = 0
Total nSV = 647
.*.*
optimization finished, #iter = 1712
obj = -749.582123, rho = -1.043393
nSV = 635, nBSV = 0
Total nSV = 635
.*.*
optimization finished, #iter = 2220
obj = -958.418907, rho = 1.092036
nSV = 785, nBSV = 1
Total nSV = 785


In [27]:
make_model(male_data_combined,female_data_combined,get_hinge_with_cold,"hinge_with_cold_combined",True,70)

[LibSVM]..*.*
optimization finished, #iter = 2516
obj = -1463.188378, rho = -0.678204
nSV = 458, nBSV = 100
Total nSV = 458
..*.*
optimization finished, #iter = 2460
obj = -1474.327054, rho = -0.746623
nSV = 452, nBSV = 99
Total nSV = 452
..*.*
optimization finished, #iter = 2336
obj = -1339.763094, rho = -0.361512
nSV = 437, nBSV = 82
Total nSV = 437
..*..*
optimization finished, #iter = 2754
obj = -1338.706320, rho = -0.632320
nSV = 457, nBSV = 77
Total nSV = 457
..*.*
optimization finished, #iter = 2423
obj = -1346.285345, rho = -0.347402
nSV = 429, nBSV = 90
Total nSV = 429
..*.*
optimization finished, #iter = 2882
obj = -1788.776404, rho = 0.603998
nSV = 537, nBSV = 125
Total nSV = 537
